## Lib

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

In [36]:
import pandas as pd

from utils.import_df import import_df
from utils.check_existing_file import check_file
from utils.img_preprocessing import get_features_target, img_preprocessing, generate_dataset

## Vars

In [22]:
csv_url = "../../storage/datas/csv/clean/cleaned_dataset_with_features_top_10_species.csv"
img_directory = "/home/guillaume/Téléchargements/mushroom-dataset/clean_dataset/"

## 1 - import DF

### Full Dataset

In [23]:
# Full Dataset
df = import_df(csv_url)

DF loaded with shape: (15200, 15)
Unique species in DF: 11


In [24]:
df["species"].value_counts()

species
Pleurotus ostreatus      1623
Trametes versicolor      1608
Pluteus cervinus         1470
other                    1400
Lepista nuda             1356
Psilocybe zapotecorum    1350
Psilocybe cyanescens     1310
Boletus edulis           1303
Hypholoma fasciculare    1293
Ganoderma applanatum     1281
Galerina marginata       1206
Name: count, dtype: int64

### Sample

In [25]:
# 10% Sample
df_sample_10 = import_df(csv_url,
                         sample = True,
                         sample_size = 0.1,
                         random_state=1)

DF sampled with 10.0% from original dataset, shape: (1520, 15)
Unique species in sampled DF: 11


In [26]:
df_sample_10["species"].value_counts()

species
Pleurotus ostreatus      169
Trametes versicolor      156
other                    145
Lepista nuda             142
Boletus edulis           136
Hypholoma fasciculare    135
Ganoderma applanatum     135
Pluteus cervinus         133
Psilocybe cyanescens     124
Psilocybe zapotecorum    123
Galerina marginata       122
Name: count, dtype: int64

## 2 - Check if all image files exist

In [27]:
check_file(img_directory = img_directory,
           df = df)

All files found!


In [29]:
check_file(img_directory = img_directory,
           df = df_sample_10)

All files found!


## 3 - Df column selection

In [30]:
target_column = "species"
img_link_column = "image_lien"

In [31]:
df = df[[target_column, img_link_column]]
df_sample_10 = df_sample_10[[target_column, img_link_column]]

In [33]:
df["image_lien"] = df["image_lien"].apply(lambda x: os.path.join(img_directory, x))
df_sample_10["image_lien"] = df_sample_10["image_lien"].apply(lambda x: os.path.join(img_directory, x))

## 4 - img Preprocessing

### Features & Targets

In [37]:
# Get features & target
df_features, df_target = get_features_target(df, target_column_name = "species")
df_sample_10_features, df_sample_10_target = get_features_target(df_sample_10, target_column_name = "species")

### Img Load & Preprocessing

In [ ]:
image_preprocessing() # Terminer fonction et tester

### Tensorflow dataset generation

In [ ]:
# Construction des datasets
from fonctions_dataset import gen_dataset, load_image
dataset_train = gen_dataset(set = 'train', training_data = training_data, img_dim = img_dim, augment = True, normalize = False, batch_size = batch_size)
dataset_val = gen_dataset(set = 'validation', training_data = training_data, img_dim = img_dim, augment = False, normalize = False, batch_size = batch_size)
dataset_test = gen_dataset(set = 'test', training_data = training_data, img_dim = img_dim, augment = False, normalize = False, batch_size = batch_size)

In [ ]:
generate_dataset() # Terminer fonction et tester